In [44]:
def genome_graphs():
    import json
    import glob
    import os
    import pandas as pd
    import numpy as np

    with open("adjacency_list_dags/1000genome.json", "r") as f:
        adjacency_list = json.load(f)

    counter = 0
    edge_index = []
    lookup = {}
    for l in adjacency_list:
        lookup[l] = counter
        counter+=1

    for l in adjacency_list:
        for e in adjacency_list[l]:
            edge_index.append([lookup[l], lookup[e]])
            
    total_columns = ['type', 'is_clustered', 'ready',
    'submit', 'execute_start', 'execute_end', 'post_script_start',
    'post_script_end', 'wms_delay', 'pre_script_delay', 'queue_delay',
    'runtime', 'post_script_delay', 'stage_in_delay', 'stage_out_delay',
    'stage_in_bytes', 'stage_out_bytes', 'kickstart_user', 'kickstart_site', 
    'kickstart_hostname', 'kickstart_transformations', 'kickstart_executables',
    'kickstart_executables_argv', 'kickstart_executables_cpu_time', 'kickstart_status',
    'kickstart_executables_exitcode']
    graphs = []
    classes = {"normal": 0}
    counter = 1
    for d in os.listdir("data"):
        d = d.split("_")[0]
        if d in classes: continue
        classes[d] = counter
        counter += 1
  
    import math
    count =0
    for d in os.listdir("data"):
        for f in glob.glob(os.path.join("data", d, "1000genome*")):
            if f in ["data/cpu_3/1000genome-20200613T072602Z-run0011.csv",
                    "data/hdd_100/1000genome-20200901T234910Z-run0048.csv",
                    "data/hdd_100/1000genome-20200901T234910Z-run0049.csv",
                    "data/loss_1/1000genome-20200520T215721Z-run0014.csv",
                    "data/norma/1000genome-20200616T174351Z-run0022.csv", 
                    "data/normal/1000genome-20200616T174351Z-run0022.csv"]: continue

            print(f)

            graph = {"y": classes[d.split("_")[0]], "edge_index": edge_index, "x":[]}
            features = pd.read_csv(f, index_col=[0])
            # time_list
            for l in lookup:
                if l.startswith("create_dir_") or l.startswith("cleanup_"):
                    new_l = l.split("-")[0]
                else:
                    new_l = l

                job_features = features[features.index.str.startswith(new_l)][total_columns].values.tolist()[0]

                job_features = [-1 if x != x else x for x in job_features]   
                minim = min(job_features[2:8])
                job_features[2:8] = [ j-minim for j in job_features[2:8]]
                graph['x'].insert(lookup[l], job_features)

            graphs.append(graph)
            return graphs, edge_index

graphs = genome_graphs

In [45]:
print(len(graphs))
## Dumped into the pickle file.
import pickle
with open('graph_genome_preprocessed.pkl','wb') as f:
    pickle.dump(graphs, f)

1270


In [46]:
## Dumped into the pickle file.
import pickle
with open('graph_genome_preprocessed.pkl','rb') as f:
    graphs = pickle.load(f)
print(len(graphs))

1270


In [47]:
number_data=[]
flat_list = [item  for sublist in graphs for item in sublist['x']]
number_data = [len(sublist['x']) for sublist in graphs]
print(len(flat_list), len(number_data))
df = pd.DataFrame(flat_list, columns=['type', 'is_clustered', 'ready',
                                    'submit', 'execute_start', 'execute_end', 'post_script_start',
                                    'post_script_end', 'wms_delay', 'pre_script_delay', 'queue_delay',
                                    'runtime', 'post_script_delay', 
                                    'stage_in_bytes', 'stage_out_bytes', 'kickstart_user', 'kickstart_site', 
                                    'kickstart_hostname', 'kickstart_transformations',
                                    'kickstart_executables_argv', 'kickstart_executables_cpu_time', 'kickstart_status',
                                    'kickstart_executables_exitcode'])


from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for string in ['type', 'kickstart_user', 'kickstart_site', 'kickstart_hostname','kickstart_executables',\
                   'kickstart_transformations']:
        df[string] =labelencoder.fit_transform(df[string])
df['kickstart_executables_argv'] = labelencoder.fit_transform(df['kickstart_executables_argv'].astype('str'))
df.head()
df.tail()

72390 1270


ValueError: 23 columns passed, passed data had 24 columns

In [31]:
array = df.to_numpy().astype('float64')

In [32]:
prev = 0
nexts= 0
new_min, new_max = 0, 1
gx = array
v_min, v_max = gx.min(), gx.max()
gx = (gx - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
print(gx.min(), gx.max())
# from sklearn.preprocessing import StandardScaler
# scaler =  StandardScaler()
# scaler.fit_transform(gx)
for i in range(len(graphs)):
    nexts+=number_data[i]
    graphs[i]['x']= array[prev:nexts,:]
    prev+=number_data[i]
    #print("i", i, prev, nexts, graphs[i]['x'].shape)
#print(len(graphs))

0.0 1.0


In [33]:
## Dumped into the pickle file.
import pickle
with open('graph_processed.pkl','wb') as f:
    pickle.dump(graphs, f)

In [34]:
## Genome transformation on the data
## Dumped into the pickle file.
import pickle
with open('graph_processed.pkl','rb') as f:
    graphs = pickle.load(f)

In [35]:
import numpy as np
y_list = []
for gr in graphs:
    y_list.append(gr['y'])
print(min(y_list))
print(max(y_list))
print(np.unique(np.array(y_list), return_counts=True))


0
3
(array([0, 1, 2, 3]), array([199, 249, 573, 249]))


In [36]:
import torch
from torch_geometric.data import Data
datasets=[]
import numpy
from sklearn.preprocessing import StandardScaler
scaler =  StandardScaler()
for element in graphs:
    gx = torch.from_numpy(element['x'] )
    ge =torch.tensor(numpy.array(element['edge_index']) ).T
    gy =torch.tensor(numpy.array(element['y']).reshape([-1]))
    if gx.shape[0] >0 :
        datasets.append( Data(x=gx, edge_index=ge, y=gy) )

In [37]:
import torch
from torch_geometric.datasets import TUDataset

dataset = datasets

# print()
# print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')
data = dataset[0]  # Get the first graph object.
print()
print(data)
print('=============================================================')
# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of graphs: 1270

Data(x=[57, 24], edge_index=[2, 129], y=[1])
Number of nodes: 57
Number of edges: 129
Average node degree: 2.26
Has isolated nodes: True
Has self-loops: False
Is undirected: False


In [38]:
torch.manual_seed(12345)
import random
random.seed(12345)
random.shuffle(datasets)
train_dataset = datasets[:1000]
test_dataset = datasets[1000:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 1000
Number of test graphs: 270


In [39]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 256
DataBatch(x=[14592, 24], edge_index=[2, 33024], y=[256], batch=[14592], ptr=[257])

Step 2:
Number of graphs in the current batch: 256
DataBatch(x=[14592, 24], edge_index=[2, 33024], y=[256], batch=[14592], ptr=[257])

Step 3:
Number of graphs in the current batch: 256
DataBatch(x=[14592, 24], edge_index=[2, 33024], y=[256], batch=[14592], ptr=[257])

Step 4:
Number of graphs in the current batch: 232
DataBatch(x=[13224, 24], edge_index=[2, 29928], y=[232], batch=[13224], ptr=[233])



In [42]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,  SAGEConv
from torch_geometric.nn import global_mean_pool
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(24, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 4)
        

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # 3. Apply a final classifier
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.0001, training=self.training)
        x = self.lin2(x)
        return x

In [43]:
# from IPython.display import Javascript
# display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64).float()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x.float(), data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0            
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x.float(), data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


import torch.optim.lr_scheduler as lrs
# scheduler = lrs.ExponentialLR(optimizer, gamma=0.99)
for epoch in range(1, 1000):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch%5==0:
        #scheduler.step()
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 005, Train Acc: 0.4450, Test Acc: 0.4741
Epoch: 010, Train Acc: 0.1920, Test Acc: 0.2111
Epoch: 015, Train Acc: 0.1920, Test Acc: 0.2111
Epoch: 020, Train Acc: 0.4450, Test Acc: 0.4741
Epoch: 025, Train Acc: 0.1610, Test Acc: 0.1444


KeyboardInterrupt: 